#### Data Section

Wrangling data from two sources, where the neighbourhoods of interest are from New York. 

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import json # library to handle JSON files

#### Let us become familiar with the data. This data was obtained from Kaggle datasets 

NY City data : [https://www.kaggle.com/new-york-city/nyc-property-sales#nyc-rolling-sales.csv](https://www.kaggle.com/new-york-city/nyc-property-sales#nyc-rolling-sales.csv) for house prices and the data we obtain from 

2014 New York City Neighborhood Names: https://geo.nyu.edu/catalog/nyu_2451_34572

In [2]:
# Load the NYC Property data to a pandas dataframe object 
nyc_prop_data2 = pd.read_csv("nyc-property-sales/nyc-rolling-sales.csv", index_col=0) 

In [3]:
# Wrangle data from a JSON file 
with open('nyu_2451_34572-geojson.json') as json_data:
    newyork_data = json.load(json_data)

neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
nyc_prop_data = pd.DataFrame(columns=column_names)


for data in neighborhoods_data:
    borough = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    nyc_prop_data = nyc_prop_data.append({'PostalCode': borough,
                                    'Borough':neighborhood_name,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
nyc_prop_data.set_index("PostalCode",inplace=True)
nyc_prop_data.head()

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
Bronx,Wakefield,Wakefield,40.894705,-73.847201
Bronx,Co-op City,Co-op City,40.874294,-73.829939
Bronx,Eastchester,Eastchester,40.887556,-73.827806
Bronx,Fieldston,Fieldston,40.895437,-73.905643
Bronx,Riverdale,Riverdale,40.890834,-73.912585


#### In this data we can see that it has the longitude, latitude and neighbourhood column these should come in handy for Foursquare. 

In [4]:
nyc_prop_data.dtypes

Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [5]:
nyc_prop_data.Neighborhood = [string.upper() for string in nyc_prop_data.Neighborhood.values]

#### Some data wrangling. We merge data from NYC Properties and from the json file 

In [6]:
nyc_data = nyc_prop_data2[['NEIGHBORHOOD','SALE PRICE']]
nyc_data.rename(columns={'SALE PRICE':'PRICE', 'NEIGHBORHOOD': 'Neighborhood'}, inplace=True)

In [7]:
print(nyc_data.shape)

(84548, 2)


#### We remove the dashes i.e. - from the Property dataset and we merge this with NYC neighbourhoods names data we do this in order to enable foursquare to get a more polished dataset with relevant CITY names.

In [8]:
nyc_data.head(10)

,Neighborhood,PRICE
4,ALPHABET CITY,6625000
5,ALPHABET CITY,-
6,ALPHABET CITY,-
7,ALPHABET CITY,3936272
8,ALPHABET CITY,8000000
9,ALPHABET CITY,-
10,ALPHABET CITY,3192840
11,ALPHABET CITY,-
12,ALPHABET CITY,-
13,ALPHABET CITY,16232000


In [51]:
merge_df =pd.merge(nyc_data,
                   nyc_prop_data,
                   how="inner",
                   on='Neighborhood')

In [52]:
merge_df.shape

(57599, 5)

In [53]:
merge_df.head(10)

,Neighborhood,PRICE,Borough,Latitude,Longitude
0,CHELSEA,-,Chelsea,40.744035,-74.003116
1,CHELSEA,-,Chelsea,40.594726,-74.189560
2,CHELSEA,-,Chelsea,40.744035,-74.003116
3,CHELSEA,-,Chelsea,40.594726,-74.189560
4,CHELSEA,7425000,Chelsea,40.744035,-74.003116
5,CHELSEA,7425000,Chelsea,40.594726,-74.189560
6,CHELSEA,10,Chelsea,40.744035,-74.003116
7,CHELSEA,10,Chelsea,40.594726,-74.189560
8,CHELSEA,10,Chelsea,40.744035,-74.003116
9,CHELSEA,10,Chelsea,40.594726,-74.189560


In [57]:
merge_df.drop(['Borough'], axis=1, inplace=True)

#### More data cleaning

In [58]:
merge_df.replace(' -  ', np.nan, inplace=True)
merge_df.dropna(inplace=True)
merge_df.PRICE = merge_df.PRICE.astype(np.float64)

#### Aggregations: Group by average price

In [59]:
df_grp_price = merge_df.groupby(['Neighborhood', 'Latitude', 'Longitude'])['PRICE'].mean().reset_index()

In [60]:
df_grp_price.head()

,Neighborhood,Latitude,Longitude,PRICE
0,ANNADALE,40.538114,-74.178549,6.280466e+05
1,ARDEN HEIGHTS,40.549286,-74.185887,3.949565e+05
2,ARROCHAR,40.596313,-74.067124,5.700459e+05
3,ARVERNE,40.589144,-73.791992,3.794519e+05
4,ASTORIA,40.768509,-73.915654,1.399695e+06


#### This is where the data section ends!

#### We can either save data to a CSV file or just re-run this process for the Methodology section. 
